In [9]:
import numpy as np
import numba
import quantecon as qe
import matplotlib.pyplot as plt
import datetime
import interpolation

In [10]:
beta=0.96
alpha=0.36
delta=0.08
a_max = 25.
b=3.

In [11]:
#rho=0.2
rho=0.4

#mu=0
mu=3.
#mu=5

#sigma=0
sigma=0.4
#sigma=0.6
#sigma=0.9

In [12]:
x = qe.markov.approximation.tauchen(rho,sigma*(1-rho**2)**0.5, m = 3,n = 7)
l_grid = np.exp(x.state_values)
l_dis = x.stationary_distributions
L = np.sum(l_grid * l_dis)
l_min=l_grid[0]
piel = x.P

In [40]:
def plc_iter(phi, beta, mu, trans, r, w, a_grid, l_grid):
    
    cons = np.empty((601,7), dtype=np.float64)
    a_temp = np.empty((601,7), dtype=np.float64)
    drn = np.ones((601,7), dtype=np.float64)*12.
    
    n=0
    err=1.    
    
    while err > 1e-5 and n <= 1000:
        dr0=drn.copy()
        cons = (beta*(1+r) * ((drn)**(-mu)) @ trans) ** (-1/mu)
        a_temp = (cons + a_grid - w*l_grid)/(1+r)
        for i in range(7):
            location = np.where(a_grid<=a_temp[0,i])
            if location[0].shape[0]==0:
                sup = 0
            else:
                sup = np.max(location[0])+1
            drn[sup:601,i] = interpolation.interp(a_temp[:,i], cons[:,i], a_grid[sup:601,0])
            drn[0:sup,i] = (1+r)*a_grid[0:sup,0] + w*l_grid[0,i] + phi
    
        if n % 5 == 0:
            err = np.max(np.abs(drn-dr0))
        n+=1
    
    fin = (1+r)*a_grid + w*l_grid - drn
    return fin

In [5]:
@numba.njit
def plc_iter(phi, beta, mu, trans, r, w, a_grid, l_grid):
    
    cons = np.empty((601,7), dtype=np.float64)
    a_temp = np.empty((601,7), dtype=np.float64)
    drn = np.ones((601,7), dtype=np.float64)*12.
    
    n=0
    err=1.    
    
    while err > 1e-5 and n <= 1000:
        dr0=drn.copy()
        cons = (beta*(1+r) * ((drn)**(-mu)) @ trans) ** (-1/mu)
        a_temp = (cons + a_grid - w*l_grid)/(1+r)
        for i in range(7):
            location = np.where(a_grid<=a_temp[0,i])
            if location[0].shape[0]==0:
                sup = 0
            else:
                sup = np.max(location[0])+1
            drn[sup:601,i] = interpolation.interp(a_temp[:,i], cons[:,i], a_grid[sup:601,0])
            drn[0:sup,i] = (1+r)*a_grid[0:sup,0] + w*l_grid[0,i] + phi
    
        if n % 5 == 0:
            err = np.max(np.abs(drn-dr0))
        n+=1
    
    fin = (1+r)*a_grid + w*l_grid - drn
    return fin

In [14]:
def demo_iter(a_grid, plc, pie):
    err = 1.
    n = 0
    
    demon = np.ones((601,7), dtype=np.float64) / (601*7)
    itv = a_grid[1:601]-a_grid[0:600]
    
    while err > 1e-6 and n <= 1000:
        demo = demon.copy()
        temp = np.zeros((601*7,8), dtype=np.float64)
        demon = np.zeros((601,7), dtype=np.float64)
        for i in range(601):
            for j in range(7):
                temp[i*7+j,0]=plc[i,j]
                temp[i*7+j,1:8]=demo[i,j] * pie[j,:]
        
        for k in range(600):
            location = np.where(np.logical_and(np.greater_equal(temp[:,0],a_grid[k]), np.less_equal(temp[:,0],a_grid[k+1])))
            l = location[0]
            demon[k,:] = np.sum(temp[l,1:8]*(np.reshape((temp[l,0]-a_grid[k]), (-1,1))/itv[k]), axis=0) + demon[k,:]
            demon[k+1,:] = np.sum(temp[l,1:8]*(np.reshape((a_grid[k+1]-temp[l,0]), (-1,1))/itv[k]), axis=0) + demon[k+1,:]
        
        if n % 10 == 0:        
            err = np.max(np.abs(demon - demo))
            
        n += 1
        
    return demon

In [6]:
@numba.njit
def demo_iter(a_grid, plc, pie):
    err = 1.
    n = 0
    
    demon = np.ones((601,7), dtype=np.float64) / (601*7)
    itv = a_grid[1:601]-a_grid[0:600]
    
    while err > 1e-6 and n <= 1000:
        demo = demon.copy()
        temp = np.zeros((601*7,8), dtype=np.float64)
        demon = np.zeros((601,7), dtype=np.float64)
        for i in range(601):
            for j in range(7):
                temp[i*7+j,0]=plc[i,j]
                temp[i*7+j,1:8]=demo[i,j] * pie[j,:]
        
        for k in range(600):
            location = np.where(np.logical_and(np.greater_equal(temp[:,0],a_grid[k]), np.less_equal(temp[:,0],a_grid[k+1])))
            l = location[0]
            demon[k,:] = np.sum(temp[l,1:8]*(np.reshape((temp[l,0]-a_grid[k]), (-1,1))/itv[k]), axis=0) + demon[k,:]
            demon[k+1,:] = np.sum(temp[l,1:8]*(np.reshape((a_grid[k+1]-temp[l,0]), (-1,1))/itv[k]), axis=0) + demon[k+1,:]
        
        if n % 10 == 0:        
            err = np.max(np.abs(demon - demo))
            
        n += 1
        
    return demon

In [41]:
err = 1.
count = 0
r = 0.01

#momentum = 0.15
#lr = 0.001
#decay = 0.98
r_low=-0.04
r_up=(1-beta)/beta

start = datetime.datetime.now()

l_grid_re = np.reshape(l_grid, (1,-1))
trans = np.transpose(piel)

while err >= 15e-4 and count <= 30:
    k = (((r+delta)/(alpha * L**(1-alpha)))) ** (1/(alpha-1))
    w = (1-alpha) * (k**alpha) * (L**(-alpha))
    
    if r > 0:
        phi = min(b, w*l_min/r)
    else:
        phi = b
    
    a_grid = np.linspace(-phi, a_max, 601, dtype=np.float64)
    a_grid_re = np.reshape(a_grid, (-1,1))
    plc = plc_iter(phi, beta, mu, trans, r, w, a_grid_re, l_grid_re)
    
    demo = demo_iter(a_grid, plc, piel)
    kprime = np.sum(demo*a_grid_re)
    
    err = np.abs(k - kprime)
    if kprime < k:
        r_low = r
    else:
        r_up = r
    r = 0.5*(r_up+r_low)
    
    if count % 10 ==0:
        print(count)
    
    count += 1

end = datetime.datetime.now()
print(end-start)

0
10
0:02:24.871735


In [16]:
k, kprime, r, w

(6.390075882618307, 6.388724934722765, 0.03617273966471357, 1.2091523735138896)

In [42]:
k, kprime, r, w

(6.291828871221992, 6.292147120252405, 0.03732564290364587, 1.2024265355324681)

In [13]:
w

1.0773325810396563

In [11]:
r

0.06263229728210717

In [36]:
test=np.where(a_grid<-6)

In [37]:
np.max(test[0])

ValueError: zero-size array to reduction operation maximum which has no identity

In [39]:
test[0].shape[0]

0